# IPL PP Score Prediction

In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import joblib
X_raw = pd.read_csv('all_matches.csv', low_memory=False)

In [2]:
X_raw.columns

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'penalty', 'wicket_type', 'player_dismissed', 'other_wicket_type',
       'other_player_dismissed'],
      dtype='object')

In [3]:
X_raw.shape

(200664, 22)

## Data Preprocessing

In [4]:
# # Selecting useful features
# X_train = pd.DataFrame()
# X_train = X_raw.loc[:,['match_id','venue','innings','batting_team','bowling_team','ball','runs_off_bat','extras','striker','non_striker','bowler']]
# X_train

# # Kings XI Punjab to Punjab Kings and Delhi Daredevils to Delhi Capitals
# mapping = {
#     'Kings XI Punjab' : 'Punjab Kings',
#     'Delhi Daredevils' : 'Delhi Capitals',
#     'Kolkata Knight Riders' : 'Kolkata Knight Riders',
#     "Delhi Capitals" : "Delhi Capitals",
#     "Mumbai Indians" : "Mumbai Indians",
#     "Chennai Super Kings":"Chennai Super Kings",
#     "Rajasthan Royals":"Rajasthan Royals",
#     "Royal Challengers Bangalore":"Royal Challengers Bangalore",
#     "Sunrisers Hyderabad":"Sunrisers Hyderabad"
# }
# X_train.batting_team = X_train.batting_team.map(mapping)
# X_train.bowling_team = X_train.bowling_team.map(mapping)

# # Mapping venues
# mapping2 = {
#     "Wankhede Stadium":"Wankhede Stadium",
#     "Wankhede Stadium, Mumbai":"Wankhede Stadium",
#     "MA Chidambaram Stadium, Chepauk":"MA Chidambaram Stadium",
#     "MA Chidambaram Stadium":"MA Chidambaram Stadium",
#     "MA Chidambaram Stadium, Chepauk, Chennai":"MA Chidambaram Stadium",
#     "M Chinnaswamy Stadium":"M.Chinnaswamy Stadium",
#     "M.Chinnaswamy Stadium":"M.Chinnaswamy Stadium",
#     "Eden Gardens":"Eden Gardens",
#     "Feroz Shah Kotla":"Arun Jaitley Stadium",
#     "Arun Jaitley Stadium":"Arun Jaitley Stadium",
#     "Sardar Patel Stadium, Motera":"Narendra Modi Stadium"
# }
# X_train.venue = X_train.venue.map(mapping2)

# # Adding new feature: Totalruns 
# X_train['Totalruns'] = X_train['runs_off_bat'] + X_train['extras']

# # Selecting first six overs and removing superover innings
# X_train1 = X_train.loc[(X_train.ball < 6.1) & (X_train.innings.isin([1,2]))]

# # Removing old teams and unwanted venues
# teams=["Delhi Capitals","Kolkata Knight Riders","Mumbai Indians","Chennai Super Kings","Sunrisers Hyderabad","Rajasthan Royals","Punjab Kings","Royal Challengers Bangalore"]
# venues=["Wankhede Stadium","MA Chidambaram Stadium","M.Chinnaswamy Stadium","Eden Gardens","Arun Jaitley Stadium","Narendra Modi Stadium"]
# X_train1=X_train1.loc[(X_train1.batting_team.isin(teams))]
# X_train1=X_train1.loc[(X_train1.bowling_team.isin(teams))]
# X_train1=X_train1.loc[(X_train1.venue.isin(venues))]

# # Grouping by matches,innings,etc and creating dataframe for storing totalruns, batsmen list and bowler list
# df = X_train1.groupby(['match_id','innings','venue','batting_team','bowling_team'])

# c1=df.Totalruns.sum()
# c2=df["striker"].unique()
# c3=df["bowler"].unique()

# df1=c1.to_frame(name = 'Totalruns').reset_index()
# df2=c2.to_frame(name = 'batsmen').reset_index()
# df3=c3.to_frame(name = 'bowlers').reset_index()

# dff=df1.merge(df2, how='right', on=['match_id','venue','innings','batting_team','bowling_team'])
# data=dff.merge(df3, how='right', on=['match_id','venue','innings','batting_team','bowling_team'])

# #Removing this row as its a mistake
# index_names = data[ data['Totalruns'] == 2 ].index
# data.drop(index_names, inplace = True)

# # Removing match_id as it is just an index
# del data["match_id"]
# data.reset_index(drop=True,inplace=True)

# # Shifting Totalruns to rightmost column and interchanging pos of innings and venue: required output as per convention
# data = data[["venue", "innings", "batting_team","bowling_team","batsmen","bowlers","Totalruns"]]


# #Additional Processing for this level
# # Taking number of batsmen and no of bowlers as new columns
# data["NumBatsmen"]=data["batsmen"].apply(lambda x: len(x))
# data["NumBowlers"]=data["bowlers"].apply(lambda x: len(x))

# del data["batsmen"]
# del data["bowlers"]
# data.reset_index(drop=True,inplace=True)

# # Shifting Totalruns to rightmost column and interchanging pos of innings and venue: required output as per convention
# data = data[["venue", "innings", "batting_team","bowling_team","NumBatsmen","NumBowlers","Totalruns"]]

# # FINAL DATAFRAME
# data

In [5]:
# Selecting useful features
X_train = pd.DataFrame()
X_train = X_raw.loc[:,['match_id','venue','innings','batting_team','bowling_team','ball','runs_off_bat','extras','striker','non_striker','bowler']]
X_train

,match_id,venue,innings,batting_team,bowling_team,ball,runs_off_bat,extras,striker,non_striker,bowler
0,335982,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.1,0,1,SC Ganguly,BB McCullum,P Kumar
1,335982,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.2,0,0,BB McCullum,SC Ganguly,P Kumar
2,335982,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.3,0,1,BB McCullum,SC Ganguly,P Kumar
3,335982,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.4,0,0,BB McCullum,SC Ganguly,P Kumar
4,335982,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.5,0,0,BB McCullum,SC Ganguly,P Kumar
...,...,...,...,...,...,...,...,...,...,...,...
200659,1254086,"Narendra Modi Stadium, Ahmedabad",2,Delhi Capitals,Punjab Kings,17.2,6,0,SO Hetmyer,S Dhawan,RP Meredith
200660,1254086,"Narendra Modi Stadium, Ahmedabad",2,Delhi Capitals,Punjab Kings,17.3,6,0,SO Hetmyer,S Dhawan,RP Meredith
200661,1254086,"Narendra Modi Stadium, Ahmedabad",2,Delhi Capitals,Punjab Kings,17.4,4,0,SO Hetmyer,S Dhawan,RP Meredith
200662,1254086,"Narendra Modi Stadium, Ahmedabad",2,Delhi Capitals,Punjab Kings,17.5,0,1,SO Hetmyer,S Dhawan,RP Meredith


In [6]:
# Kings XI Punjab to Punjab Kings and Delhi Daredevils to Delhi Capitals
mapping = {
    'Kings XI Punjab' : 'Punjab Kings',
    'Delhi Daredevils' : 'Delhi Capitals',
    'Kolkata Knight Riders' : 'Kolkata Knight Riders',
    "Delhi Capitals" : "Delhi Capitals",
    "Mumbai Indians" : "Mumbai Indians",
    "Chennai Super Kings":"Chennai Super Kings",
    "Rajasthan Royals":"Rajasthan Royals",
    "Royal Challengers Bangalore":"Royal Challengers Bangalore",
    "Sunrisers Hyderabad":"Sunrisers Hyderabad"
}
X_train.batting_team = X_train.batting_team.map(mapping)
X_train.bowling_team = X_train.bowling_team.map(mapping)

# Mapping venues
mapping2 = {
    "Wankhede Stadium":"Wankhede Stadium",
    "Wankhede Stadium, Mumbai":"Wankhede Stadium",
    "MA Chidambaram Stadium, Chepauk":"MA Chidambaram Stadium",
    "MA Chidambaram Stadium":"MA Chidambaram Stadium",
    "MA Chidambaram Stadium, Chepauk, Chennai":"MA Chidambaram Stadium",
    "M Chinnaswamy Stadium":"M.Chinnaswamy Stadium",
    "M.Chinnaswamy Stadium":"M.Chinnaswamy Stadium",
    "Eden Gardens":"Eden Gardens",
    "Feroz Shah Kotla":"Arun Jaitley Stadium",
    "Arun Jaitley Stadium":"Arun Jaitley Stadium",
    "Sardar Patel Stadium, Motera":"Narendra Modi Stadium"
}
X_train.venue = X_train.venue.map(mapping2)

In [7]:
# Adding new feature: Totalruns 
X_train['Totalruns'] = X_train['runs_off_bat'] + X_train['extras'] 

In [8]:
# Selecting first six overs and removing superover innings
X_train1 = X_train.loc[(X_train.ball < 6.1) & (X_train.innings.isin([1,2]))]
X_train1

,match_id,venue,innings,batting_team,bowling_team,ball,runs_off_bat,extras,striker,non_striker,bowler,Totalruns
0,335982,M.Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.1,0,1,SC Ganguly,BB McCullum,P Kumar,1
1,335982,M.Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.2,0,0,BB McCullum,SC Ganguly,P Kumar,0
2,335982,M.Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.3,0,1,BB McCullum,SC Ganguly,P Kumar,1
3,335982,M.Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.4,0,0,BB McCullum,SC Ganguly,P Kumar,0
4,335982,M.Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.5,0,0,BB McCullum,SC Ganguly,P Kumar,0
...,...,...,...,...,...,...,...,...,...,...,...,...
200587,1254086,NaN,2,Delhi Capitals,NaN,5.2,0,0,PP Shaw,S Dhawan,CJ Jordan,0
200588,1254086,NaN,2,Delhi Capitals,NaN,5.3,6,0,PP Shaw,S Dhawan,CJ Jordan,6
200589,1254086,NaN,2,Delhi Capitals,NaN,5.4,4,0,PP Shaw,S Dhawan,CJ Jordan,4
200590,1254086,NaN,2,Delhi Capitals,NaN,5.5,1,0,PP Shaw,S Dhawan,CJ Jordan,1


In [9]:
# Removing old teams and unwanted venues
teams = ["Delhi Capitals","Kolkata Knight Riders","Mumbai Indians","Chennai Super Kings","Sunrisers Hyderabad","Rajasthan Royals","Punjab Kings","Royal Challengers Bangalore"]
venues = ["Wankhede Stadium","MA Chidambaram Stadium","M.Chinnaswamy Stadium","Eden Gardens","Arun Jaitley Stadium","Narendra Modi Stadium"]
X_train1 = X_train1.loc[(X_train1.batting_team.isin(teams))]
X_train1 = X_train1.loc[(X_train1.bowling_team.isin(teams))]
X_train1 = X_train1.loc[(X_train1.venue.isin(venues))]

In [10]:
# Grouping by matches,innings,etc and creating dataframe for storing totalruns, batsmen list and bowler list
df = X_train1.groupby(['match_id','innings','venue','batting_team','bowling_team'])
df.first()

ball  \
match_id innings venue                  batting_team                bowling_team                        
335982   1       M.Chinnaswamy Stadium  Kolkata Knight Riders       Royal Challengers Bangalore   0.1   
         2       M.Chinnaswamy Stadium  Royal Challengers Bangalore Kolkata Knight Riders         0.1   
335984   1       Arun Jaitley Stadium   Rajasthan Royals            Delhi Capitals                0.1   
         2       Arun Jaitley Stadium   Delhi Capitals              Rajasthan Royals              0.1   
335985   1       Wankhede Stadium       Mumbai Indians              Royal Challengers Bangalore   0.1   
...                                                                                               ...   
1254075  2       Wankhede Stadium       Rajasthan Royals            Kolkata Knight Riders         0.1   
1254076  1       Wankhede Stadium       Chennai Super Kings         Royal Challengers Bangalore   0.1   
         2       Wankhede Stadium       Royal Challengers Bangalore Chennai Super Kings           0.1   
1254077  1       MA Chidambaram Stadium Delhi Capitals              Sunrisers Hyderabad           0.1   
         2       MA Chidambaram Stadium Sunrisers Hyderabad         Delhi Capitals                0.1   

                                                                                                 runs_off_bat  \
match_id innings venue                  batting_team                bowling_team                                
335982   1       M.Chinnaswamy Stadium  Kolkata Knight Riders       Royal Challengers Bangalore             0   
         2       M.Chinnaswamy Stadium  Royal Challengers Bangalore Kolkata Knight Riders                   1   
335984   1       Arun Jaitley Stadium   Rajasthan Royals            Delhi Capitals                          0   
         2       Arun Jaitley Stadium   Delhi Capitals              Rajasthan Royals                        0   
335985   1       Wankhede Stadium       Mumbai Indians              Royal Challengers Bangalore             0   
...                                                                                                       ...   
1254075  2       Wankhede Stadium       Rajasthan Royals            Kolkata Knight Riders                   0   
1254076  1       Wankhede Stadium       Chennai Super Kings         Royal Challengers Bangalore             1   
         2       Wankhede Stadium       Royal Challengers Bangalore Chennai Super Kings                     4   
1254077  1       MA Chidambaram Stadium Delhi Capitals              Sunrisers Hyderabad                     4   
         2       MA Chidambaram Stadium Sunrisers Hyderabad         Delhi Capitals                          0   

                                                                                                 extras  \
match_id innings venue                  batting_team                bowling_team                          
335982   1       M.Chinnaswamy Stadium  Kolkata Knight Riders       Royal Challengers Bangalore       1   
         2       M.Chinnaswamy Stadium  Royal Challengers Bangalore Kolkata Knight Riders             0   
335984   1       Arun Jaitley Stadium   Rajasthan Royals            Delhi Capitals                    0   
         2       Arun Jaitley Stadium   Delhi Capitals              Rajasthan Royals                  0   
335985   1       Wankhede Stadium       Mumbai Indians              Royal Challengers Bangalore       0   
...                                                                                                 ...   
1254075  2       Wankhede Stadium       Rajasthan Royals            Kolkata Knight Riders             1   
1254076  1       Wankhede Stadium       Chennai Super Kings         Royal Challengers Bangalore       0   
         2       Wankhede Stadium       Royal Challengers Bangalore Chennai Super Kings               0   
1254077  1       MA Chidambaram Stadium Delhi Capitals              Sunrisers Hydera

In [11]:
c1 = df.Totalruns.sum()
c2 = df["striker"].unique()
c3 = df["bowler"].unique()

df1 = c1.to_frame(name = 'Totalruns').reset_index()
df2 = c2.to_frame(name = 'batsmen').reset_index()
df3 = c3.to_frame(name = 'bowlers').reset_index()

dff = df1.merge(df2, how='right', on=['match_id','venue','innings','batting_team','bowling_team'])
data = dff.merge(df3, how='right', on=['match_id','venue','innings','batting_team','bowling_team'])

data

,match_id,innings,venue,batting_team,bowling_team,Totalruns,batsmen,bowlers
0,335982,1,M.Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,61,"[SC Ganguly, BB McCullum, RT Ponting]","[P Kumar, Z Khan, AA Noffke]"
1,335982,2,M.Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,26,"[R Dravid, W Jaffer, V Kohli, JH Kallis, CL Wh...","[AB Dinda, I Sharma, AB Agarkar]"
2,335984,1,Arun Jaitley Stadium,Rajasthan Royals,Delhi Capitals,40,"[T Kohli, YK Pathan, SR Watson, M Kaif]","[GD McGrath, B Geeves, MF Maharoof]"
3,335984,2,Arun Jaitley Stadium,Delhi Capitals,Rajasthan Royals,55,"[G Gambhir, V Sehwag, S Dhawan]","[MM Patel, SR Watson, SK Trivedi]"
4,335985,1,Wankhede Stadium,Mumbai Indians,Royal Challengers Bangalore,47,"[L Ronchi, ST Jayasuriya, DJ Thornely, RV Utha...","[P Kumar, Z Khan, JH Kallis]"
...,...,...,...,...,...,...,...,...
664,1254075,2,Wankhede Stadium,Rajasthan Royals,Kolkata Knight Riders,50,"[JC Buttler, YBK Jaiswal, SV Samson, S Dube]","[Shivam Mavi, PJ Cummins, CV Varun, SP Narine]"
665,1254076,1,Wankhede Stadium,Chennai Super Kings,Royal Challengers Bangalore,51,"[RD Gaikwad, F du Plessis]","[Mohammed Siraj, KA Jamieson, YS Chahal, NA Sa..."
666,1254076,2,Wankhede Stadium,Royal Challengers Bangalore,Chennai Super Kings,65,"[V Kohli, D Padikkal, Washington Sundar, GJ Ma...","[DL Chahar, SM Curran, SN Thakur]"
667,1254077,1,MA Chidambaram Stadium,Delhi Capitals,Sunrisers Hyderabad,51,"[PP Shaw, S Dhawan]","[KK Ahmed, Abhishek Sharma, S Kaul, J Suchith]"


In [12]:
#Removing this row as its a mistake

index_names = data[ data['Totalruns'] == 2 ].index 
# data[ data['Totalruns'] < 10 ].value_counts -->  # Here is one data, which shows that total runs in PP is only '2' !!
data.drop(index_names, inplace = True)

# Removing match_id as it is just an index
del data["match_id"]
data.reset_index(drop=True, inplace=True)

In [13]:
data

,innings,venue,batting_team,bowling_team,Totalruns,batsmen,bowlers
0,1,M.Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,61,"[SC Ganguly, BB McCullum, RT Ponting]","[P Kumar, Z Khan, AA Noffke]"
1,2,M.Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,26,"[R Dravid, W Jaffer, V Kohli, JH Kallis, CL Wh...","[AB Dinda, I Sharma, AB Agarkar]"
2,1,Arun Jaitley Stadium,Rajasthan Royals,Delhi Capitals,40,"[T Kohli, YK Pathan, SR Watson, M Kaif]","[GD McGrath, B Geeves, MF Maharoof]"
3,2,Arun Jaitley Stadium,Delhi Capitals,Rajasthan Royals,55,"[G Gambhir, V Sehwag, S Dhawan]","[MM Patel, SR Watson, SK Trivedi]"
4,1,Wankhede Stadium,Mumbai Indians,Royal Challengers Bangalore,47,"[L Ronchi, ST Jayasuriya, DJ Thornely, RV Utha...","[P Kumar, Z Khan, JH Kallis]"
...,...,...,...,...,...,...,...
663,2,Wankhede Stadium,Rajasthan Royals,Kolkata Knight Riders,50,"[JC Buttler, YBK Jaiswal, SV Samson, S Dube]","[Shivam Mavi, PJ Cummins, CV Varun, SP Narine]"
664,1,Wankhede Stadium,Chennai Super Kings,Royal Challengers Bangalore,51,"[RD Gaikwad, F du Plessis]","[Mohammed Siraj, KA Jamieson, YS Chahal, NA Sa..."
665,2,Wankhede Stadium,Royal Challengers Bangalore,Chennai Super Kings,65,"[V Kohli, D Padikkal, Washington Sundar, GJ Ma...","[DL Chahar, SM Curran, SN Thakur]"
666,1,MA Chidambaram Stadium,Delhi Capitals,Sunrisers Hyderabad,51,"[PP Shaw, S Dhawan]","[KK Ahmed, Abhishek Sharma, S Kaul, J Suchith]"


In [14]:
data.columns

Index(['innings', 'venue', 'batting_team', 'bowling_team', 'Totalruns',
       'batsmen', 'bowlers'],
      dtype='object')

In [15]:
# Shifting Totalruns to rightmost column and interchanging pos of innings and venue: required output as per convention
data = data[["venue", "innings", "batting_team", "bowling_team", "batsmen", "bowlers", "Totalruns"]]

In [16]:
# Additional Processing for this level
# Taking number of batsmen and no of bowlers as new columns

data["NumBatsmen"] = data["batsmen"].apply(lambda x: len(x))
data["NumBowlers"] = data["bowlers"].apply(lambda x: len(x))

del data["batsmen"]
del data["bowlers"]

data.reset_index(drop=True, inplace=True)

C:\Users\Dell\AppData\Local\Temp/ipykernel_13780/1730203922.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["NumBatsmen"] = data["batsmen"].apply(lambda x: len(x))
C:\Users\Dell\AppData\Local\Temp/ipykernel_13780/1730203922.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["NumBowlers"] = data["bowlers"].apply(lambda x: len(x))


In [17]:
# Shifting Totalruns to rightmost column and interchanging pos of innings and venue: required output as per convention
data = data[["venue", "innings", "batting_team","bowling_team","NumBatsmen","NumBowlers","Totalruns"]]

# FINAL DATAFRAME
data

,venue,innings,batting_team,bowling_team,NumBatsmen,NumBowlers,Totalruns
0,M.Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,3,3,61
1,M.Chinnaswamy Stadium,2,Royal Challengers Bangalore,Kolkata Knight Riders,6,3,26
2,Arun Jaitley Stadium,1,Rajasthan Royals,Delhi Capitals,4,3,40
3,Arun Jaitley Stadium,2,Delhi Capitals,Rajasthan Royals,3,3,55
4,Wankhede Stadium,1,Mumbai Indians,Royal Challengers Bangalore,5,3,47
...,...,...,...,...,...,...,...
663,Wankhede Stadium,2,Rajasthan Royals,Kolkata Knight Riders,4,4,50
664,Wankhede Stadium,1,Chennai Super Kings,Royal Challengers Bangalore,2,4,51
665,Wankhede Stadium,2,Royal Challengers Bangalore,Chennai Super Kings,4,3,65
666,MA Chidambaram Stadium,1,Delhi Capitals,Sunrisers Hyderabad,2,4,51


## Encoding of categorical inputs and feature scaling

In [18]:
X = data.iloc[:,:-1]  # remove last column 'Totalruns'
y = data["Totalruns"]

In [19]:
X

,venue,innings,batting_team,bowling_team,NumBatsmen,NumBowlers
0,M.Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,3,3
1,M.Chinnaswamy Stadium,2,Royal Challengers Bangalore,Kolkata Knight Riders,6,3
2,Arun Jaitley Stadium,1,Rajasthan Royals,Delhi Capitals,4,3
3,Arun Jaitley Stadium,2,Delhi Capitals,Rajasthan Royals,3,3
4,Wankhede Stadium,1,Mumbai Indians,Royal Challengers Bangalore,5,3
...,...,...,...,...,...,...
663,Wankhede Stadium,2,Rajasthan Royals,Kolkata Knight Riders,4,4
664,Wankhede Stadium,1,Chennai Super Kings,Royal Challengers Bangalore,2,4
665,Wankhede Stadium,2,Royal Challengers Bangalore,Chennai Super Kings,4,3
666,MA Chidambaram Stadium,1,Delhi Capitals,Sunrisers Hyderabad,2,4


In [173]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(categories="auto"), [0,1,2,3])], remainder='passthrough')
Xenc = pd.DataFrame(ct.fit_transform(X).toarray())
sc = StandardScaler()
Xenc = sc.fit_transform(Xenc)

In [174]:
Xenc.shape

(668, 26)

## Train-test split

In [175]:
X = Xenc
y = y

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

## Helper function

In [176]:
def evaluate(X,y):
    y_pred = np.round(regressor.predict(X))
    np.set_printoptions(precision=2)
    rmse = np.sqrt(np.mean((y-y_pred)**2))
    print("RMSE: ",rmse)

## Linear Regression

In [177]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)
print("Train: ",end=" ")
evaluate(X_train,y_train)
print("Test: ",end=" ")
evaluate(X_test,y_test)

Train:  RMSE:  10.570877243298396
Test:  RMSE:  10.220698927527618


In [178]:
model_enc=[regressor,ct,sc]
joblib.dump(model_enc, 'model1.pkl')

['model1.pkl']

## Decision Tree

In [179]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor()
regressor.fit(X_train, y_train)
print("Train: ",end=" ")
evaluate(X_train,y_train)
print("Test: ",end=" ")
evaluate(X_test,y_test)

Train:  RMSE:  3.5439984612909976
Test:  RMSE:  13.726812198552452


In [180]:
model_enc=[regressor,ct,sc]
joblib.dump(model_enc, 'model2.pkl')

['model2.pkl']

## Random Forest Regression

In [181]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor()
regressor.fit(X_train, y_train)
print("Train: ",end=" ")
evaluate(X_train,y_train)
print("Test: ",end=" ")
evaluate(X_test,y_test)

Train:  RMSE:  5.362513804841542
Test:  RMSE:  11.517993710038118


In [182]:
model_enc=[regressor,ct,sc]
joblib.dump(model_enc, 'model3.pkl')

['model3.pkl']

## KNN

In [183]:
from sklearn.neighbors import KNeighborsRegressor
regressor = KNeighborsRegressor()
regressor.fit(X_train, y_train)
print("Train: ",end=" ")
evaluate(X_train,y_train)
print("Test: ",end=" ")
evaluate(X_test,y_test)

Train:  RMSE:  10.096352291199777
Test:  RMSE:  11.855472947376251


In [184]:
model_enc=[regressor,ct,sc]
joblib.dump(model_enc, 'model4.pkl')

['model4.pkl']

## SVR

In [189]:
from sklearn.svm import SVR
regressor = SVR()
regressor.fit(X_train, y_train)
print("Train: ",end=" ")
evaluate(X_train,y_train)
print("Test: ",end=" ")
evaluate(X_test,y_test)

Train:  RMSE:  10.910943549417565
Test:  RMSE:  10.73041429977315


In [190]:
model_enc=[regressor,ct,sc]
joblib.dump(model_enc, 'model5.pkl')

['model5.pkl']

## XGBoost

In [191]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Han\AppData\Local\Programs\Python\Python39\pythonw.exe -m pip install --upgrade pip' command.


In [192]:
import xgboost as xgb
regressor = xgb.XGBRegressor()
regressor.fit(X_train, y_train)
print("Train: ",end=" ")
evaluate(X_train,y_train)
print("Test: ",end=" ")
evaluate(X_test,y_test)

Train:  RMSE:  3.89684401258595
Test:  RMSE:  13.073557681356716


In [193]:
model_enc=[regressor,ct,sc]
joblib.dump(model_enc, 'model6.pkl')

['model6.pkl']